# Segmenting and Clustering Neighborhoods in Paris

## My project description

<div style="margin-top: 20px">

<font size = 2>
In my Capstone project I want to study the different tourist places by neighborhood  of Paris, the city of lights. Study the different small activities around each tourist place such as restaurants, libraries, etc. Also study the correlation between the population by neighborhood and the number of restaurants and their type.
 
</font>
</div>

### Data source

To do this project, I retrieve the data from the opendata site https://opendata.paris.fr/page/home/, by doing web scraping with beautifulsoup, or dowloading the data in a csv file or using the restful API.

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install lxml
import lxml

     |████████████████████████████████| 112kB 7.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.8MB 7.5MB/s eta 0:00:01


In [2]:
!wget  https://opendata.paris.fr/explore/dataset/quartier_paris/download?format=csv -O paris_quartiers.csv

--2020-02-13 16:41:03--  https://opendata.paris.fr/explore/dataset/quartier_paris/download?format=csv
Resolving opendata.paris.fr (opendata.paris.fr)... 34.249.199.226, 34.248.20.69
Connecting to opendata.paris.fr (opendata.paris.fr)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘paris_quartiers.csv’

paris_quartiers.csv     [  <=>               ] 340.17K  1.06MB/s    in 0.3s    

2020-02-13 16:41:05 (1.06 MB/s) - ‘paris_quartiers.csv’ saved [348339]



In [3]:
df_data = pd.read_csv('paris_quartiers.csv',sep=';')
df_data.head()

,n_sq_qu,c_qu,c_quinsee,l_qu,c_ar,n_sq_ar,perimetre,surface,geom_x_y,geom
0,750000015,15,7510403,Arsenal,4,750000004,2878.559656,4.872649e+05,"48.851585175,2.36476795387","{""type"": ""Polygon"", ""coordinates"": [[[2.368512..."
1,750000018,18,7510502,Jardin-des-Plantes,5,750000005,4052.729521,7.983894e+05,"48.8419401934,2.35689388962","{""type"": ""Polygon"", ""coordinates"": [[[2.364561..."
2,750000039,39,7511003,Porte-Saint-Martin,10,750000010,3245.891413,6.090347e+05,"48.8712446509,2.36150364735","{""type"": ""Polygon"", ""coordinates"": [[[2.363917..."
3,750000043,43,7511103,Roquette,11,750000011,4973.010557,1.172087e+06,"48.8570640408,2.38036406173","{""type"": ""Polygon"", ""coordinates"": [[[2.379720..."
4,750000046,46,7511202,Picpus,12,750000012,18261.910318,7.205014e+06,"48.8303592424,2.42882681508","{""type"": ""Polygon"", ""coordinates"": [[[2.411249..."


## Data analylis and cleaning data for using it

In [5]:

paris_data = df_data.loc[:,['c_ar','geom_x_y','c_quinsee','l_qu']]
paris_data['c_quinsee'] = paris_data['c_quinsee'].astype(str)
paris_data['code'] = paris_data['c_quinsee'].str[0:5].astype(str)
paris_data['Latitude'] = paris_data['geom_x_y'].str.extract('(.*)\,')
paris_data['Longitude'] = paris_data['geom_x_y'].str.extract(',(.*)')
paris_data = paris_data.drop(['geom_x_y','c_quinsee'],axis=1)
paris_data.columns= ['Borough', 'Neighbourhood', 'Code', 'Latitude','Longitude']
#paris_data = paris_data.reset_index()

paris_data['Borough'] = paris_data['Borough'].astype(str)
paris_data['Code'] = paris_data['Code'].astype(str)
paris_data['Latitude'] = paris_data['Latitude'].astype(float)
paris_data['Longitude'] = paris_data['Longitude'].astype(float)

paris_data.head()

,Borough,Neighbourhood,Code,Latitude,Longitude
0,4,Arsenal,75104,48.851585,2.364768
1,5,Jardin-des-Plantes,75105,48.841940,2.356894
2,10,Porte-Saint-Martin,75110,48.871245,2.361504
3,11,Roquette,75111,48.857064,2.380364
4,12,Picpus,75112,48.830359,2.428827


## Load data of paris's borough population

In [5]:
page_url  ='https://fr.wikipedia.org/wiki/Arrondissements_de_Paris'

response = requests.get(page_url)

if response is not None:
    soup = BeautifulSoup(response.text, 'html.parser')


In [6]:
#soup.prettify()
table = soup.find("table", {"class":"wikitable sortable alternance jquery-tablesorter"})
print(table)

None


# Displaying Paris's Map

In [11]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [12]:
# create map of Paris using latitude and longitude values
longitude = 2.3488

latitude = 48.85341
 
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, codepost, neighborhood, borough in zip(paris_data['Latitude'], paris_data['Longitude'], paris_data['Code'], paris_data['Neighbourhood'], paris_data['Borough']):
    label = '{},{}, {} ème'.format(codepost, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris) 

map_paris

# Using Foursquare data to load Paris's venues 

Set credentials

In [14]:
CLIENT_ID = 'AAXS11JUTRNPPC4LYAOQYLTGYUAH13CIHKHXHJEQJY4ERUDM' # your Foursquare ID
CLIENT_SECRET = '0QDSFHVBM3Z1DPVSSWXMCESYJNHNPPJMRT4OJUGZV3IFNDC0' # your Foursquare Secret
VERSION = '20200210' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AAXS11JUTRNPPC4LYAOQYLTGYUAH13CIHKHXHJEQJY4ERUDM
CLIENT_SECRET:0QDSFHVBM3Z1DPVSSWXMCESYJNHNPPJMRT4OJUGZV3IFNDC0


Infos for the first dataframe's rows

In [15]:
neighborhood_latitude = paris_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = paris_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = paris_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Arsenal are 48.851585175, 2.36476795387.


Build Foursquare URL with geo coordinates

In [16]:

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Load data from Foursquare

In [19]:
results = requests.get(url).json()
#results

Display result keys in json file

In [22]:
results['response']['groups'][0].keys()

dict_keys(['type', 'name', 'items'])

Function that extracts the category of the venue

In [24]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Normalizing json of venues

In [25]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sherry Butt,Cocktail Bar,48.853267,2.364114
1,Maria Loca,Cocktail Bar,48.852123,2.365667
2,Pavillon de l'Arsenal,Museum,48.850650,2.362340
3,Keep Cool,Gym,48.852085,2.363371
4,Le Temps des Cerises,French Restaurant,48.852554,2.364195


List of Categories

In [27]:
nearby_venues['categories'].unique()

array(['Cocktail Bar', 'Museum', 'Gym', 'French Restaurant',
       'Vegetarian / Vegan Restaurant', 'Tapas Restaurant', 'Pizza Place',
       'Beer Store', 'Spa', 'Pedestrian Plaza', 'Wine Bar', 'Park',
       'Garden', 'Plaza', 'Thai Restaurant',
       'Southwestern French Restaurant', 'Boat or Ferry',
       'Italian Restaurant', 'Pub', 'Gastropub', 'Opera House', 'Bakery',
       'Hotel', 'Brazilian Restaurant', 'Historic Site', 'Gourmet Shop',
       'Music Venue', 'Restaurant', 'Supermarket', 'Seafood Restaurant',
       'Coffee Shop', 'Camera Store', 'Canal Lock', 'Tea Room', 'Bar',
       'Bagel Shop', 'Ice Cream Shop', 'Bistro', 'Burger Joint',
       'Szechuan Restaurant'], dtype=object)

Number of venues from Foursquare

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

69 venues were returned by Foursquare.
